In [1]:
import pandas as pd 
import json
import sqlglot
from sqlglot import exp
import oracledb
from dotenv import load_dotenv
import os
import re

#### 1. 쿼리 데이터 불러오기

In [2]:
df = pd.DataFrame()

In [3]:
# 데이터 출처 : https://github.com/glee4810/ehrsql-2024/tree/master/data/mimic_iv

with open(r"C:\last_project\mimic_query_data\mimic_iv\train\label.json", "r", encoding='utf-8') as f:
    train = json.load(f)

with open(r"C:\last_project\mimic_query_data\mimic_iv\valid\label.json", "r", encoding='utf-8') as f:
    valid = json.load(f)

with open(r"C:\last_project\mimic_query_data\mimic_iv\test\label.json", "r", encoding='utf-8') as f:
    test = json.load(f)

In [4]:
for i in [train, valid, test]:
    l = [[j] for j in list(i.values())]
    x = pd.DataFrame(l, columns=['쿼리'])
    df = pd.concat([df,x])

df.head()

,쿼리
0,SELECT DISTINCT prescriptions.route FROM presc...
1,SELECT DISTINCT prescriptions.route FROM presc...
2,SELECT DISTINCT prescriptions.route FROM presc...
3,SELECT DISTINCT prescriptions.route FROM presc...
4,SELECT DISTINCT prescriptions.route FROM presc...


In [5]:
# null 값이면 제외하기
df = df[(~ df['쿼리'].isnull()) & (~ df['쿼리'].str.contains('null'))]

#### 2. sqlite 쿼리를 oracle 쿼리로 변환

In [6]:
def change_oracle_query(sqlite_query):
    try:
        oracle_query = sqlglot.transpile(sqlite_query, read="sqlite", write="oracle")[0]
        if sqlglot.parse_one(oracle_query, read='oracle'):
            return oracle_query
    except:
        pass

In [7]:
df['oracle_query'] = df['쿼리'].apply(change_oracle_query)

In [8]:
df = df[~df['oracle_query'].isnull()]

In [9]:
df.shape

(5418, 2)

In [10]:
# 쿼리문에서 바인드 변수로 치환할 수 있으면 치환하기
def change_bind_query(query):
    try:
        parsed = sqlglot.parse_one(query, dialect="oracle")

        bind_values = {}
        bind_idx = 1

        for literal in parsed.find_all(exp.Literal):
            parent = literal.parent
            
            if parent and "format" in parent.args and parent.args["format"] is literal:
                continue

            bind_name = f"v{bind_idx}"
            bind_idx += 1

            if literal.is_string:
                val = literal.this  # 실제 문자열 값

                # YYYY-MM 형태인지 체크 (안전)
                if (
                    isinstance(val, str)
                    and len(val) >= 7
                    and val[:4].isdigit()
                    and val[4] == "-"
                ):
                    year = int(val[:4])

                    if year % 1000 == 2:
                        new_year = year - 188
                        bind_values[bind_name] = f"{new_year}{val[4:]}"
                    else:
                        bind_values[bind_name] = val
                else:
                    bind_values[bind_name] = val

            elif literal.is_number:
                bind_values[bind_name] = float(literal.this)

            # ⭐ Literal → Parameter 로 교체
            literal.replace(exp.Parameter(this=bind_name))

        res = parsed.sql(dialect="oracle").replace("@",":")
        return res
    except:
        return "invalid bind"

In [11]:
df['bind_query'] = df['oracle_query'].apply(change_bind_query)

In [12]:
import pandas as pd
df = pd.read_csv('바인드쿼리로변환한거포함.csv')

In [13]:
# 바인드 변수로 치환해서 sql 형태가 같으면 하나만 남기고 드롭하기 
# 5000개 넘어가는 쿼리에서 최종적으로 1097개 남음
new_df = df.drop_duplicates(subset=['bind_query'], keep='first').reset_index(drop=True)
new_df.head()

,쿼리,oracle_query,bind_query
0,SELECT DISTINCT prescriptions.route FROM presc...,SELECT DISTINCT prescriptions.route FROM presc...,SELECT DISTINCT prescriptions.route FROM presc...
1,SELECT DISTINCT cost.cost FROM cost WHERE cost...,SELECT DISTINCT cost.cost FROM cost WHERE cost...,SELECT DISTINCT cost.cost FROM cost WHERE cost...
2,SELECT DISTINCT cost.cost FROM cost WHERE cost...,SELECT DISTINCT cost.cost FROM cost WHERE cost...,SELECT DISTINCT cost.cost FROM cost WHERE cost...
3,SELECT DISTINCT cost.cost FROM cost WHERE cost...,SELECT DISTINCT cost.cost FROM cost WHERE cost...,SELECT DISTINCT cost.cost FROM cost WHERE cost...
4,SELECT patients.gender FROM patients WHERE pat...,SELECT patients.gender FROM patients WHERE pat...,SELECT patients.gender FROM patients WHERE pat...


#### 3. 오라클 DB 연결하기

In [14]:
load_dotenv('.env')

oracledb.init_oracle_client(lib_dir=r"C:\instant_client\instantclient_21_19")

conn = oracledb.connect(
    user=os.getenv('user'),
    password=os.getenv('password'),
    dsn=os.getenv('ORACLE_DSN')
)
cur = conn.cursor()

In [15]:
# 기존 날짜에 맞추기 위해 기존 데이터에서 188년을 빼서 이거 맞추는 함수

def fix_year_in_query(sql):
    """2xxx년을 1xxx년으로 변환"""
    # 모든 'YYYY-MM' 찾아서 변환
    matches = re.findall(r"'(\d{4})-(\d{2})'", sql)
    
    for year_str, month_str in matches:
        year = int(year_str)
        if year >= 2026:  # 2002, 2102, 3002 등
            new_year = year - 188
            old_str = f"'{year_str}-{month_str}'"
            new_str = f"'{new_year}-{month_str}'"
            sql = sql.replace(old_str, new_str)
    
    return sql

#### 4. 인덱스가 얼마나 사용되었는지 카운트하기

In [21]:
from collections import Counter

def collect_index_statistics(table_name, initial):
    table_query = new_df[new_df['bind_query'].str.contains(table_name)]
    report = []

    for i in range(len(table_query)):
        used_indexes = []
        try:
            sql = 'EXPLAIN PLAN FOR ' + fix_year_in_query(table_query['oracle_query'].iloc[i])
            cur.execute(sql)
            cur.execute("SELECT * FROM TABLE(DBMS_XPLAN.DISPLAY)")
            plan_text = "\n".join(r[0] for r in cur.fetchall())

            for line in plan_text.splitlines():
                if initial in line and '|' in line:
                    used_indexes.append(line.strip())

            report.append({
                "query_idx": i,
                "used_indexes": "; ".join(used_indexes),
                "plan": plan_text,
            })
        except:
            continue

    x = pd.DataFrame(report)
    
    for_idx_count = []
    for n in range(len(x)):
        find_pre_index = [i for i in x['used_indexes'].iloc[n].split(';') if 'INDEX' in i]
        for index_info in find_pre_index:
            for_idx_count += [i.strip() for i in index_info.split('|')][2:4]
    
    for_idx_count = [i for i in for_idx_count if 'INDEX' in i and 'SCAN' in i]
    return len(x), Counter(for_idx_count)

In [22]:
# labevents 테이블에서 인덱스가 얼마나 활용되었는가
# 총 실행된 82개의 쿼리에서 다음과 같이 인덱스가 잘 활용됨을 볼 수 있음 
collect_index_statistics('labevents', 'LAB')

(82,
 Counter({'INDEX UNIQUE SCAN': 52,
          'INDEX RANGE SCAN': 14,
          'INDEX FULL SCAN': 3,
          'INDEX FAST FULL SCAN': 3}))

In [23]:
collect_index_statistics("prescriptions", "PRES")

(79, Counter({'INDEX RANGE SCAN': 108}))

In [24]:
collect_index_statistics("chartevents", "CE")

(107, Counter({'INDEX RANGE SCAN': 126}))

In [25]:
collect_index_statistics("inputevents", "IE")

(64, Counter({'INDEX RANGE SCAN': 65}))